In [203]:
import pandas as pd
import tweepy       # python wrapper for twitter api
import json
import time
import requests
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# step 0, get your own twitter credentials!

In [204]:
# this is my own personal twitter api information
# if you could be so kind as to sign up yourself on both twitter and mashape that'd be great :)
# It's FREEEEEEE
api_key = 'NAw1wegVEZueRDqyA27c0pjjT'
api_secret = '0IbLwj3CPnsMs6xDPrRtjLDPvLAuYT1US6bgM5pvSlRemIFwrl'
access_token = '3278767399-BzpJJJxsA64HQqxPHRrjIaoNaPrz0mxfsOnl2Qu'
access_secret = 'Q2B7TbCNkZUTEsXfmFtpx9GlnMk3SimvW7WKmJC2waV0o'

In [205]:
# Documentation is your friend! http://docs.tweepy.org/en/v3.1.0/
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth) # returns a tweepy authorization handler object

In [206]:
# 1. Use Twitter API to stream and store tweets (300 per candidate)
# 37.781157,-122.39872 is the lat,lng for SF
hillary_tweets = api.search(q='#ImWithHer', count=100, geocode="37.781157,-122.398720,10mi") + api.search(q='#hillaryclinton', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#hillary2016', count=100, geocode="37.781157,-122.398720,50mi")
the_donald_tweets = api.search(q='#donaldtrump', count=100, geocode="37.781157,-122.398720,10mi") + api.search(q='#trump2016', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#trump', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#makeamericagreatagain', count=100, geocode="37.781157,-122.398720,50mi")

In [207]:
the_donald_tweets[0].text

u'RT @EdUyeshima: #DonaldTrump shows how to manipulate #TaxCodes to avoid paying taxes. Does that make him unfit to be #USPresident https://t\u2026'

In [208]:
hillary_tweets[0].text

u'#JusticeforTati #ImWithHer #Tatianna https://t.co/PSoRFbJKyD'

In [209]:
# wrappers come with built in python attributes and methods!
print dir(the_donald_tweets[0])

['__class__', '__delattr__', '__dict__', '__doc__', '__eq__', '__format__', '__getattribute__', '__getstate__', '__hash__', '__init__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_api', '_json', 'author', 'contributors', 'coordinates', 'created_at', 'destroy', 'entities', 'favorite', 'favorite_count', 'favorited', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'metadata', 'parse', 'parse_list', 'place', 'retweet', 'retweet_count', 'retweeted', 'retweeted_status', 'retweets', 'source', 'source_url', 'text', 'truncated', 'user']


In [210]:
print the_donald_tweets[0].created_at
print the_donald_tweets[0].text
print the_donald_tweets[0].favorite_count
print the_donald_tweets[0].favorited
print the_donald_tweets[0].lang
print the_donald_tweets[0].geo
print the_donald_tweets[0].retweet_count
print the_donald_tweets[0].retweeted
print the_donald_tweets[0].coordinates

2016-10-03 05:46:01
RT @EdUyeshima: #DonaldTrump shows how to manipulate #TaxCodes to avoid paying taxes. Does that make him unfit to be #USPresident https://t…
0
False
en
None
1
False
None


In [211]:
# modify the tweets to make a nice dictionary
def parse_status_objects(tweet_list):
    modified_tweets = list()
    for x in tweet_list:
        modified_tweets.append( [
                    x.created_at,
                    x.text,
                    x.favorite_count,
                    x.favorited,
                    x.lang,
                    x.geo,
                    x.retweet_count,
                    x.retweeted,
                    x.coordinates
                ])
    return modified_tweets
        
hillary_tweets = parse_status_objects(hillary_tweets)
the_donald_tweets = parse_status_objects(the_donald_tweets)

In [212]:
print the_donald_tweets[0]
print hillary_tweets[0]

[datetime.datetime(2016, 10, 3, 5, 46, 1), u'RT @EdUyeshima: #DonaldTrump shows how to manipulate #TaxCodes to avoid paying taxes. Does that make him unfit to be #USPresident https://t\u2026', 0, False, u'en', None, 1, False, None]
[datetime.datetime(2016, 10, 3, 5, 48, 20), u'#JusticeforTati #ImWithHer #Tatianna https://t.co/PSoRFbJKyD', 0, False, u'und', None, 0, False, None]


In [213]:
# 2 Create a dataframe that combines all of the tweets from each candidate
# To do so, you will need to concat the data frames
# so each row is a tweet and your columns should be 
# date, text, favorite_count,favorited, language, geocode, retweet count, retweeted, coordinates

In [214]:
donald_df = pd.DataFrame(the_donald_tweets)
donald_df.rename(columns={0:'date', 1:'text', 2:'favorite_count',
                         3:'favorited', 4:'language', 5:'geocode',
                         6:'retweet_count', 7: 'retweeted',
                         8:'coordinates' }, inplace=True)

In [215]:
hillary_df = pd.DataFrame(hillary_tweets)
hillary_df.rename(columns={0:'date', 1:'text', 2:'favorite_count',
                         3:'favorited', 4:'language', 5:'geocode',
                         6:'retweet_count', 7: 'retweeted',
                         8:'coordinates' }, inplace=True)

In [216]:
all_tweets = pd.concat([hillary_df, donald_df])

In [217]:
all_tweets.head()

,date,text,favorite_count,favorited,language,geocode,retweet_count,retweeted,coordinates
0,2016-10-03 05:48:20,#JusticeforTati #ImWithHer #Tatianna https://t...,0,False,und,None,0,False,None
1,2016-10-03 05:47:07,RT @Robrt_M_Goldste: #ImWithHer #Propaganda #L...,0,False,en,None,1,False,None
2,2016-10-03 05:40:42,#ImWithHer From a gaming point of view #Trump'...,1,False,en,None,0,False,None
3,2016-10-03 05:31:30,#ImWithHer #Propaganda #Lies #GOP #WordPress #...,0,False,en,None,1,False,None
4,2016-10-03 05:21:06,#ImWithHer 'learning' how stupid one is comes ...,0,False,en,None,0,False,None


In [218]:
all_tweets.shape

(682, 9)

In [219]:
# 3 Create a function to take in a string and output the textblob sentiment of that string

In [220]:
from textblob import TextBlob, Word

def detect_sentiment(text):
    return round(TextBlob(text).sentiment.polarity,3)

In [221]:
detect_sentiment('this is not good')

-0.35

In [222]:
detect_sentiment('this is awesome')

1.0

In [223]:
# 4 Add a column to your data frame called 'sentiment'
# which holds the sentiment of that tweet (hint: use the function from #3)

In [224]:
all_tweets['sentiment'] = all_tweets.text.apply(detect_sentiment)

In [225]:
all_tweets

,date,text,favorite_count,favorited,language,geocode,retweet_count,retweeted,coordinates,sentiment
0,2016-10-03 05:48:20,#JusticeforTati #ImWithHer #Tatianna https://t...,0,False,und,None,0,False,None,0.000
1,2016-10-03 05:47:07,RT @Robrt_M_Goldste: #ImWithHer #Propaganda #L...,0,False,en,None,1,False,None,0.018
2,2016-10-03 05:40:42,#ImWithHer From a gaming point of view #Trump'...,1,False,en,None,0,False,None,0.000
3,2016-10-03 05:31:30,#ImWithHer #Propaganda #Lies #GOP #WordPress #...,0,False,en,None,1,False,None,0.018
4,2016-10-03 05:21:06,#ImWithHer 'learning' how stupid one is comes ...,0,False,en,None,0,False,None,-0.800
5,2016-10-03 05:17:34,#ImWithHer Poor Rudy...It must be hard to be a...,0,False,en,None,0,False,None,-0.346
6,2016-10-03 05:02:01,.@JamilSmith Enthusiasm is something I think o...,0,False,en,None,0,False,None,-0.333
7,2016-10-03 04:58:46,#ImWithHer the most ridiculously stupid commen...,0,False,en,None,0,False,None,0.033
8,2016-10-03 04:45:00,"""Because if basketball has taught me anything,...",1,False,en,None,1,False,None,0.000
9,2016-10-03 04:32:02,#Trump #ImWithHer #Propaganda #Fox #GOP #CNN #...,0,False,fr,None,0,False,None,-0.050


In [226]:
all_tweets.text[6]

6    .@JamilSmith Enthusiasm is something I think o...
6    This twat .@StephenMoore thinks that evading t...
Name: text, dtype: object

In [227]:
# 5 create a word count column, which holds the number of words in the text column

In [228]:
def word_count_in_text(text):
    return len(text.split())

In [229]:
word_count_in_text('hello how you')

3

In [230]:
all_tweets['word_count'] = all_tweets.text.apply(word_count_in_text)

In [231]:
all_tweets.head()

,date,text,favorite_count,favorited,language,geocode,retweet_count,retweeted,coordinates,sentiment,word_count
0,2016-10-03 05:48:20,#JusticeforTati #ImWithHer #Tatianna https://t...,0,False,und,None,0,False,None,0.000,4
1,2016-10-03 05:47:07,RT @Robrt_M_Goldste: #ImWithHer #Propaganda #L...,0,False,en,None,1,False,None,0.018,19
2,2016-10-03 05:40:42,#ImWithHer From a gaming point of view #Trump'...,1,False,en,None,0,False,None,0.000,21
3,2016-10-03 05:31:30,#ImWithHer #Propaganda #Lies #GOP #WordPress #...,0,False,en,None,1,False,None,0.018,17
4,2016-10-03 05:21:06,#ImWithHer 'learning' how stupid one is comes ...,0,False,en,None,0,False,None,-0.800,20


In [232]:
# use countvectorizer to create a document-term matrix and concatinate it to your main dataframe
#(hint you should now have thousands of columns)
# Don't add count vectorizer result to the original dataframe

In [233]:
vect = CountVectorizer()
words_tweets = pd.DataFrame(vect.fit_transform(all_tweets.text).toarray(), columns=vect.get_feature_names())

In [234]:
words_tweets.shape

(682, 2463)

In [235]:
words_tweets.head()

,000,00am,00pm,100,11,12,13,13t2cskvef,155,155mph,...,yxn5wh6xcu,yybsgoiq5q,zbutn0z6js,zero,zie2y6m6dk,zkygeys9kr,zmoy4dbinj,zones,zp4tsxosic,zxlkaelt92
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [236]:
all_tweets.shape

(682, 11)

In [237]:
words_tweets.shape

(682, 2463)

In [240]:
combined = pd.merge(all_tweets,words_tweets, right_index=True, left_index=True)

In [241]:
combined.head()

,date,text_x,favorite_count,favorited,language_x,geocode,retweet_count,retweeted_x,coordinates,sentiment,...,yxn5wh6xcu,yybsgoiq5q,zbutn0z6js,zero,zie2y6m6dk,zkygeys9kr,zmoy4dbinj,zones,zp4tsxosic,zxlkaelt92
0,2016-10-03 05:48:20,#JusticeforTati #ImWithHer #Tatianna https://t...,0,False,und,None,0,False,None,0.000,...,0,0,0,0,0,0,0,0,0,0
0,2016-10-03 05:46:01,RT @EdUyeshima: #DonaldTrump shows how to mani...,0,False,en,None,1,False,None,0.000,...,0,0,0,0,0,0,0,0,0,0
1,2016-10-03 05:47:07,RT @Robrt_M_Goldste: #ImWithHer #Propaganda #L...,0,False,en,None,1,False,None,0.018,...,0,0,0,0,0,0,0,0,0,0
1,2016-10-03 05:29:56,#DonaldTrump shows how to manipulate #TaxCodes...,0,False,en,None,1,False,None,0.000,...,0,0,0,0,0,0,0,0,0,0
2,2016-10-03 05:40:42,#ImWithHer From a gaming point of view #Trump'...,1,False,en,None,0,False,None,0.000,...,0,0,0,0,0,0,0,0,0,0


In [242]:
combined.shape

(682, 2474)

In [154]:
# 6 Use kmean to create clusters for the dataframe (choose an optimal k)
# Don't add clusters to the original dataframe

In [247]:
from sklearn.cluster import KMeans

In [258]:
km = KMeans(n_clusters=2)

In [259]:
km.fit(words_tweets)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=2, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [261]:
words_tweets['cluster'] = km.labels_
words_tweets.sort('cluster')

C:\Users\Tim\Anaconda2\envs\datascience\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,000,00am,00pm,100,11,12,13,13t2cskvef,155,155mph,...,yybsgoiq5q,zbutn0z6js,zero,zie2y6m6dk,zkygeys9kr,zmoy4dbinj,zones,zp4tsxosic,zxlkaelt92,cluster
577,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
599,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
547,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
517,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
518,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
519,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
520,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
564,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
522,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
563,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 7 Explore the clusters
# What is interesting? Surprising? Can you draw any inferences about each cluster?
# Write your answer to these questions in markdown

In [263]:
words_tweets.groupby('cluster').mean()

,000,00am,00pm,100,11,12,13,13t2cskvef,155,155mph,...,yxn5wh6xcu,yybsgoiq5q,zbutn0z6js,zero,zie2y6m6dk,zkygeys9kr,zmoy4dbinj,zones,zp4tsxosic,zxlkaelt92
cluster,,,,,,,,,,,,,,,,,,,,,
0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.0016,0.0016,0.0016,0.0128,0.0032,0.0032,0.0016,0.0016,0.0016,0.0016,...,0.0016,0.0048,0.0016,0.0048,0.0016,0.0016,0.0016,0.0016,0.0016,0.0016


In [ ]:
# 8 Create a column called 'democrat' and assign it to the corresponding candidate name for all the records
# 1 for hilary and 0 for donald trump
# Make a boxplot of sentiment for each candidate by the "democrat" column

In [ ]:
# 9 Estimate classification models using knn and logistic regession.  Dont forget to use cross validation 
    # (do not use cross_val_score because we will want a confusion matrix).
# The cateogrical response here is predict democrat or republican
# Your features can EITHER be your countvectorizer document term matrix OR 
# data from the original dataframe (sentiment, word count, etc)

In [ ]:
# 10 Initialize a classification model (for both knn and logistic)
# This is separate from step 9 because I will eventually want a 
# confusion matrix and we can't do that via cros validation

In [ ]:
# 11 Split your data into training and testing tests and then
# train your classification model  (for both knn and logistic)

In [ ]:
# 12 Predict on your test features and targets (for both knn and logistic)

In [ ]:
# 13 output your accuracy (for both knn and logistic)

In [ ]:
# 14 Create a confusion matrix for your predictions (for both knn and logistic)
# What is the sensitivity and specificity?

In [ ]:
# 15 Who is the easiest candidate to predict? Why do you think? 
# Who is the hardest? Why do you think?
# Write your answers in markdown (for both knn and logistic)

In [ ]:
# 16 BONUS: make a pipeline with the first step being a feature union of countvectorizer
# and tfidfvectorizer and no other features and our second step is
# a ML model (your choice) predict our binary response


In [16]:
# 17: BONUS: grid search the model in step 16 to obtain the best features (up to you)

In [ ]:
# 18: SUPER BONUS: make a custom transformer that extracts sentiment and word count
# make a pipeline that combines countvec, tfidfvec and your custom transformer
# and try to get the possible accuracy for predicting whether or not the person is talking
# about democrats or republicans